I worked with this notbeook inside the development data folder, so if you would like to run the code, download it and put it in the right folder (or change the paths in the notebook).

In [1]:
import pprint
import json
import codecs
from nltk.tree import Tree
from collections import defaultdict

In [2]:
# Specify path to data to be analyzed (I entered the path to the tutorial data for now, so that the code runs)

with codecs.open ('relations.json', 'r', encoding = 'utf-8') as pdtb_file:
    

    # Assign all relations (a list) to a variable
    relations = [json.loads(x) for x in pdtb_file];

In [3]:
# Loop through relations and select only explicit relations:

# List for explicit relations:

relations_explicit = []

for relation in relations:
    if relation['Type'] == 'Explicit':
        relations_explicit.append(relation)
        
print(len(relations_explicit))
pprint.pprint(relations_explicit[0])

14722
{'Arg1': {'CharacterSpanList': [[825, 961]],
          'RawText': 'The new "social choice" fund will shun securities of '
                     'companies linked to South Africa, nuclear power and in '
                     'some cases, Northern Ireland',
          'TokenList': [[825, 828, 145, 6, 0],
                        [829, 832, 146, 6, 1],
                        [833, 834, 147, 6, 2],
                        [834, 840, 148, 6, 3],
                        [841, 847, 149, 6, 4],
                        [847, 848, 150, 6, 5],
                        [849, 853, 151, 6, 6],
                        [854, 858, 152, 6, 7],
                        [859, 863, 153, 6, 8],
                        [864, 874, 154, 6, 9],
                        [875, 877, 155, 6, 10],
                        [878, 887, 156, 6, 11],
                        [888, 894, 157, 6, 12],
                        [895, 897, 158, 6, 13],
                        [898, 903, 159, 6, 14],
                        [904, 

In [4]:
# Split in two lists according to arg1 in same sentence and arg1 in previous sentence:

relations_ss = []
relations_ps = []
relations_other = []

for relation in relations_explicit:
    sentence_id_arg1 = relation['Arg1']['TokenList'][0][3]
    sentence_id_connective = relation['Connective']['TokenList'][0][3]
    sentence_id_arg2 = relation['Arg2']['TokenList'][0][3]
    
    if sentence_id_arg1 == sentence_id_connective == sentence_id_arg2:
        relations_ss.append(relation)
    elif int(sentence_id_arg1) == int(sentence_id_connective) - 1 == int(sentence_id_arg2) -1:
        relations_ps.append(relation)
    else: 
        relations_other.append(relation)
print(len(relations_ps))
print(len(relations_ss))

4119
8893


In [5]:
# Open parse file:

with codecs.open('parses.json', 'r', encoding = 'utf8') as parse_file:
    parses = json.load(parse_file)

In [6]:


def normalize_str(text):
    """"""
    
    text = text.lstrip('(')
    
    text = text.rstrip(')')
    text_list = text.split()
    
    clean_text_list = []
    
    for word in text_list:
        word = word.strip()
        clean_text_list.append(word)
    
    clean_text = ' '.join(clean_text_list)
    
    return(clean_text)

my_rel = relations_ss[7]

arg1_str = my_rel['Arg1']['RawText']
print(arg1_str)
print(normalize_str(arg1_str))
print(normalize_str("(rtc's! Work\n didn t doesn t $50"))

and that it was his mother's birthday
and that it was his mother's birthday
rtc's! Work didn t doesn t $50


In [7]:
def get_subtrees(parsed_sentence, const_type = 0):
    """"""
    tree_nltk = Tree.fromstring(parsed_sentence)
    
    tree_height = tree_nltk.height()
    
        
    const_tuple_list = []

    for s in tree_nltk.subtrees():

        label = s.label()
        
        if (label != '``') and (label != ''):
            
            subtree_tokens_str = str(s.flatten())
            #print(type(subtree_tokens_str))
            clean_subtree_str = normalize_str(subtree_tokens_str).lstrip(label).lstrip(' ')



            if const_type != 0:
                if label == const_type:
                    const_tuple_list.append((label, clean_subtree_str))

            else:

                const_tuple_list.append((label, clean_subtree_str))



    return const_tuple_list

doc_id = my_rel['DocID']
sentence_id_arg1 = my_rel['Arg1']['TokenList'][0][3]
parsed_sentence = parses[doc_id]['sentences'][sentence_id_arg1]['parsetree']  

consts = get_subtrees(parsed_sentence)

#pprint.pprint(consts)

In [8]:
def get_data(relation, arg):
    
    """returns DocID, sentenceID, Argument raw text and sentence_token_id in relations.json"""
    
    relation_id = relation['ID']
    doc_id = relation['DocID']
    sentence_id_arg = relation[arg]['TokenList'][0][3]
    arg_str = relation[arg]['RawText']   
    
    sentence_token_id_relations = []
    
    relations_token_list = relation[arg]['TokenList']
    
    for line in relations_token_list:
        sentence_token_id = line[4]
        sentence_token_id_relations.append(sentence_token_id)
    
    return relation_id, doc_id, sentence_id_arg, arg_str, sentence_token_id_relations

my_rel = relations_ss[3]
my_relid, my_docid, my_sid, my_arg, my_st = get_data(my_rel, 'Arg1')


for rel in relations_ss:
    print(get_data(rel, 'Arg1'))

(3182, 'wsj_0204', 11, 'or receive cash from the funds', [20, 28, 29, 30, 31, 32])
(3183, 'wsj_0207', 0, 'Solo woodwind players have to be creative', [0, 1, 2, 3, 4, 5, 6])
(3184, 'wsj_0207', 0, 'Solo woodwind players have to be creative if they want to work a lot', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])
(3186, 'wsj_0207', 1, 'He commissions and splendidly interprets fearsome contemporary scores and does some conducting', [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])
(3188, 'wsj_0207', 4, 'Today, the pixie-like clarinetist has mostly dropped the missionary work', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
(3192, 'wsj_0207', 7, 'He believes in what he plays', [0, 1, 2, 3, 4, 5])
(3195, 'wsj_0207', 10, 'that his new album, "Inner Voices," had just been released, that his family was in the front row', [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34])
(3196, 'wsj_0207', 10, "and that it was his mother's birthday", [36, 37, 38, 39, 40, 41, 4

In [9]:
def parse_sentence_token_id(doc_id, sentence_id_arg, sentence_token_id_relations):
    """"""
    
    argument_id_tuples_list = []
    
    parsed_words_list = parses[doc_id]['sentences'][sentence_id_arg]['words']
    
    for number, word in enumerate(parsed_words_list):
        
        if number in sentence_token_id_relations:
        
            argument_id_tuples_list.append((number, word[0]))
        
        
    return argument_id_tuples_list

my_tuple_list = parse_sentence_token_id('wsj_1000', 22, my_st)

# Looks good, if you compare these ids to the sentence_token_ids taken from relations.json. 
# Covers the span of the argument

In [37]:
def get_arg_list(argument_tuples_list):
    
    """"""


    # check if argument is interrupted by comparing to the sentence token ids in the relations.json file
    slice_index = None
    
    for index, pair in enumerate(argument_tuples_list):
        if pair == argument_tuples_list[-1]:
            break

        next_pair = argument_tuples_list[index + 1]


        if (next_pair[0] - pair[0]) > 1:
            slice_index = index
            
            break
            
        else:
            slice_index = None
    
    argument_tokens_as_in_parsing = []
    
    for st_id, token in argument_tuples_list:
        
        argument_tokens_as_in_parsing.append(token)  
    # Get the argument as either 1 string in a list (if it is not interrupted) or as 2 strings in a list
    # if it is interrupted

    if slice_index:
        arg1_1 = ' '.join(argument_tokens_as_in_parsing[:slice_index + 1])

        arg1_2 = ' '.join(argument_tokens_as_in_parsing[slice_index + 1:])

        arg_str_list = [arg1_1, arg1_2]
    else:

        arg_str_list = [' '.join(argument_tokens_as_in_parsing)]
    
    
    return arg_str_list

get_arg_list(my_tuple_list)

['with it -LCB- program trading -RCB- , but they -LCB- the exchange']

In [99]:
# Map arguments to constituents

def constituent_structure(relation, arg):
    """"""

    # Dicts with argument as key and constituent as value (tuple: (label, constituent))
    arg_is_const = defaultdict(list)
    arg_starts_const = defaultdict(list)
    arg_in_const = defaultdict(list)
    const_part_arg = defaultdict(list)
    
    
    
    rel_id, doc_id, sentence_id_arg, arg_str, sentence_token_id_relations = get_data(relation, arg)
    
    arg_tuples_list = my_tuple_list = parse_sentence_token_id(doc_id, sentence_id_arg, sentence_token_id_relations)
    
    arg_list = get_arg_list(arg_tuples_list)


    sentence = parses[doc_id]['sentences'][sentence_id_arg]['parsetree']

    const_list = get_subtrees(sentence)


    for argument in arg_list:



        for constituent in const_list:

            label, const = constituent
    


            # Constituents the argument is part of:

            if const == argument:

                arg_is_const[argument].append((label, const))




            #elif const.startswith(argument):

                #arg_starts_const[argument].append((label, const))


            #elif argument in const:

                #arg_in_const[argument].append((label, const))



            # Constituents that are part of the argument

            if const in argument:
                const_part_arg[argument].append((label, const))
                
    return rel_id, doc_id, arg_is_const, arg_starts_const, arg_in_const, const_part_arg



In [110]:
# 

with open('ss_arg1_constituents.tsv', 'w', encoding = 'utf-8') as outfile:   
    
    for n, rel in enumerate(relations_ss[8:10]):
        #print(n)
        rel_id, doc_id, dict1, dict2, dict3, dict4 = constituent_structure(rel, 'Arg1')
        
        #print(dict4)

        if dict1:
        
            for arg, const in dict1.items():
                for l, c in const:
                    print('argument matches constituent:',l, c)
                    outfile.write('same sentence\t'+str(rel_id)+'\t'+str(doc_id)+'\t'+arg+'\t'+l+'\n')
    
        else: 
            
            
            
            arg = list(dict4.keys())
          
            
            
            # for some reason, there was an empty list in one case (which makes the program break down)
            if arg:

                arg = arg[0]
            
            

                print('\n\nthe argument: ',arg)

                const_list = list(dict4.values())

                pprint.pprint(const_list)

                const_len_list = []

                for const in const_list[0]:
                    label, c = const
                    c_list = c.split()
                    const_len_list.append((len(c_list), c, label))
                    
                
                sorted_const_len_list = sorted(const_len_list, reverse = True)
                
                longest_consts = []
                
                for n, const in enumerate(sorted_const_len_list):
                    if const == sorted_const_len_list[-1]:
                        break
                        
                    length, c, l = const
    
                    
                    if sorted_const_len_list[0][0] == length:
                    
                        longest_consts.append(const)
                    
                    

                print('the longest constituent: ',longest_consts)
                
                for const in longest_consts:
                    sorted_const_len_list.remove(const)
                    
                    rest =  arg.replace(const[1], '')
                
                #print('the rest: ',rest)

                for const in sorted_const_len_list:
                    l = const[2]
                    c = const[1].strip()
                    rest = rest.strip()
                    
                    
                   
                    if c == rest:
                        print('constituent matches rest: ',l, c)
                        break
                        

                    elif c in rest:
                        
                        print('another constituent', l, c)
                        sorted_const_len_list.remove(const)
                       





the argument:  Then he offered the second movement from Saint-Saens 's Sonata for Clarinet , a whimsical , puckish tidbit that reflected the flip side of the Stoltzman personality
[[('ADVP', 'Then'),
  ('RB', 'Then'),
  (',', ','),
  ('TO', 'to'),
  ('IN', 'that'),
  ('NP', 'he'),
  ('PRP', 'he'),
  (',', ','),
  ('NP', 'he'),
  ('PRP', 'he'),
  ('VP',
   "offered the second movement from Saint-Saens 's Sonata for Clarinet , a "
   'whimsical , puckish tidbit that reflected the flip side of the Stoltzman '
   'personality'),
  ('VBD', 'offered'),
  ('NP',
   "the second movement from Saint-Saens 's Sonata for Clarinet , a whimsical "
   ', puckish tidbit that reflected the flip side of the Stoltzman '
   'personality'),
  ('NP', 'the second movement'),
  ('DT', 'the'),
  ('JJ', 'second'),
  ('NN', 'movement'),
  ('PP', "from Saint-Saens 's Sonata"),
  ('IN', 'from'),
  ('NP', "Saint-Saens 's Sonata"),
  ('NP', "Saint-Saens 's"),
  ('NNP', 'Saint-Saens'),
  ('POS', "'s"),
  ('NNP', 'S